<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Time Series: Rolling Statistics


## Learning Objectives

**After this lesson, you will be able to:**
- Define the concepts of trend and seasonality and be able to identify them visually.
- Use boxplots to compare distributions.
- Plot time series data over time to identify large-scale trends in data.
- Investigate trends by computing simple aggregates with Pandas using the `.resample()` function.
- Compute rolling statistics with Pandas to compare data of a date to a smaller window of time.
- Utilize exponentially weighted windows to average out noise.
- Use differences to remove trends in time series data.
- Use the Pandas' `.shift()` function to create lagged features.

---

## Lesson Guide

**Time Series Rolling Statistics**
- [Trend and Seasonality](#A)
- [Aggregate Data](#B)
- [Rolling Statistics](#C)
- [Differencing a Time Series and Stationarity](#D)
- [Shifting and Lagging Time Series Data](#E)
- [Independent Practice](#F)
----

<h2><a id="A">Trend and Seasonality</a></h2>

**Question:** What constitutes a trend in data? Is linearity required for a trend?

- A trend is any long-term change in the value we are measuring. Trends may “change direction,” going from an increasing trend to a decreasing trend.

- Trends can only be measured within the scope of the data collected; there may be trends that are unmeasurable if the data are not complete.

An example of an upward trend:
![](./assets/trend-line2.png)

- When patterns repeat over *known, fixed periods* of time within a data set, we call this **seasonality**.

- A seasonal pattern exists when a series is influenced by factors related to the cyclic nature of time — i.e., time of month, quarter, year, etc. Seasonality is of a fixed and known period, otherwise it is not truly seasonality. Additionally, it must be either attributed to another factor or counted as a set of anomalous events in the data.

>  Can you think of some seasonal patterns from your own experience?


### The easiest way to visualize trends is by drawing trend lines.

In [ ]:
import pandas as pd
import numpy as np
from datetime import timedelta
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

plt.rcParams['figure.figsize'] = [15, 8]
plt.rcParams['font.size'] = 14
plt.style.use('fivethirtyeight')

# Import the data.
df = pd.read_csv('data/mapquest_google_trends.csv')

df.head()

In [ ]:
# Clean/organise the data. 
df.columns = ['WeekOf', 'Hits']
print(df.head())

ax = df.plot(title = "Interest in MapQuest Over Time")
ax.set_xlabel("Week");
ax.set_ylabel("Hits");

Let's fit a Polynomial Regression of degree 1 (i.e. a Linear Regression) to compute a coefficient and y-intercept for our line.

In [ ]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()

X = np.array(df.index).reshape(-1, 1)
y = np.array(df['Hits']).reshape(-1, 1)

lin_reg.fit(X, y)

print(lin_reg.coef_, lin_reg.intercept_)

Alternatively, we can use NumPy's [`polyfit()`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.polyfit.html) method to do this.

In [ ]:
line_coef = np.polyfit(df.index, df['Hits'], 1)
print(line_coef)

Then, define our polynomial function using the obtained coefficients. We can do this on a single dimension using NumPy's [`poly1d()`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.poly1d.html) method.

In [ ]:
polynomial = np.poly1d(line_coef)
# The intercept is ~86.59, the slope is ~0.11.

# Let's take a look at the trendline values at specific points:
print(polynomial(0))
print(polynomial(1))
print(polynomial(2))

### Now, plot our trendline over the data.

In [ ]:
# Plot the time series.
plt.plot(df.index, df['Hits']);
# Plot the least squares minimizing line.
plt.plot(df.index, polynomial(df.index));

Looks like a second-order polynomial might fit our data even better. Let's try that out.

In [ ]:
line_coef = np.polyfit(df.index,df['Hits'],2)
print(line_coef)

second_polynomial = np.poly1d(line_coef)

In [ ]:
# Plot the time series.
plt.plot(df.index, df['Hits']);
# Plot the least squares minimizing line.
plt.plot(df.index, second_polynomial(df.index));

> **Question:** Can you think of any other underlying patterns that might cause trends in time series data? What might cause seasonality in a time series?

### Guided Practice

Let's look for trends and seasonality in data made available by a German drugstore, Rossmann.

These data contain the daily sales made at the drugstore, as well as whether or not a sale or holiday affected the data.

Because we want to explore the trend over time, we will make sure to convert the `Date` column to a `datetime` object and make it the index of our DataFrame, as we did with our Apple stock data. 

Let's recall the steps for preprocessing time series data with Pandas:
* Convert time data to a `datetime` object.
* Set `datetime` to index the DataFrame.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

plt.rcParams['figure.figsize'] = [10, 8]
plt.rcParams['font.size'] = 14
plt.style.use('fivethirtyeight')

In [ ]:
pharm = pd.read_csv('data/rossmann.csv', skipinitialspace=True, low_memory=False)
pharm['Date'] = pd.to_datetime(pharm['Date'])
pharm = pharm.set_index('Date')
print("{} rows by {} columns".format(pharm.shape[0], pharm.shape[1]))
pharm.head()

This allows us to easily filter by date. Let's add a column for `Year` and `Month` based on the `datetime` index. 

In [ ]:
pharm['Year'] = pharm.index.year
pharm['Month'] = pharm.index.month

pharm['2015-05'].head()

There are more than a million sales data points in this data set, so, for some simple exploratory data analysis (EDA), we'll focus on just one store.

*Note: We explicitly add a `.copy()` to the returned DataFrame because we will be making changes to this DataFrame object later. If we didn't specify this, we will receive SettingCopyWarnings later on because Pandas will not know if we want to edit a view of the original data object (`data`) or a separate data object (`store1_data`). Without the `.copy()`, the variable `store1_data` will simply be a reference the original data object in memory.*

In [ ]:
pharm_store1 = pharm[pharm['Store'] == 1].copy()
print("{} rows by {} columns".format(pharm_store1.shape[0], pharm_store1.shape[1]))
pharm_store1.head()

### Plot the sales data.

Let's investigate whether or not promotions affect sales. For this, we'll use boxplots.

On state holidays, the store is closed (which means there are zero sales), so we need to cut those days out. (Contextual knowledge like this is always necessary to truly explain time series phenomena.)

> **Check for Understanding:** Can you think of any other special considerations we should make when tracking sales?

Now, check to see if there is a difference affecting sales on promotion days.

In [ ]:
sns.boxplot(x='Promo', y='Sales', data=pharm_store1[pharm_store1['Open']==1]);

We can see that there _is_ a difference in sales on promotion days.

Why is it important to separate out days on which the store is closed? Because there aren't any promotions on those days either, so including them will bias our sales data on days without promotions! Remember to think about the business logic in addition to analyzing the raw data.

We may also want to compare sales across days of the week:

In [ ]:
sns.boxplot(x='DayOfWeek', y='Sales', data=pharm_store1);

Lastly, we want to identify larger-scale trends in our data. How did sales change from 2014 to 2015? Were there any particularly interesting outliers in terms of sales or customer visits?

To plot the sales and customer visits over time:

In [ ]:
# Filter to days Store 1 was open.
pharm_store1_open = pharm_store1[pharm_store1['Open']==1]
pharm_store1_open[['Sales']].plot();
pharm_store1_open[['Customers']].plot();

We can see that there are large spikes of sales and customers toward the end of 2013 and 2014, leading into the first quarter of 2014 and 2015.

Let's use index filtering to filter just to 2014 changes over time. This should make it easier to identify the holiday sales bump.


In [ ]:
pharm_store1_2014 = pharm_store1_open['2014']
pharm_store1_2014[['Sales']].plot();

<h2><a id= "B">Aggregate Data</a></h2>

If we want to investigate trends over time in sales, as always, we should start by computing simple aggregates. We want to know: What were the mean and median sales in each month and year?

We can use `pharm.resample` on the whole data set and provide:
    - A parameter for the level on which to roll up to: `'D'` for day, `'W'` for week, `'M'` for month, `'A'` for year.
    - The aggregation method to perform: `mean()`, `median()`, `sum()`, etc.

In [ ]:
pharm[['Sales']].resample('A').mean()

It looks like average sales were highest in 2015. Now, let's look at the median annual sales.

In [ ]:
pharm[['Sales']].resample('A').median()

In [ ]:
pharm[['Sales']].resample('M').mean().plot();

For more information, see Pandas' `.resample`  [documentation](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.resample.html).

> **Question:** How do aggregation techniques help us sort out important insights from the noise of time series data?

<h2><a id= "C">Rolling Statistics</a></h2>

With time series, we can "roll" statistics across time. For example, the rolling mean is the mean of a moving window across time periods. To understand holidays sales in pharmacies, we don't want to compare sales data in late December with the entire month but instead with a few days immediately surrounding it. We can do this using rolling averages.

![](./assets/rolling-statistics.jpeg)


### Parameters for `rolling()` Functions

Pandas offers a variety of functionalities for creating rolling statistics, which we'll only scratch the surface of here.  We'll be using a `rolling()` function with parameters `window` and `center` and a statistical function chained to it. Let's dive into more detail.

[`rolling()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rolling.html) can take the following parameters:

* `window` indicates the number of observations used for calculating the statistic.
* `center` indicates whether the window should be centered on the date or use data prior to that date.


##### Calculate the rolling daily sum over all stores.

Use the `.resample()` function to calculate the daily total over all of the stores.

In [ ]:
daily_pharm_sales = pharm[['Sales']].resample('D').sum()
daily_pharm_sales.head()

Use the `.rolling()` function to calculate the rolling average over a three-day period.

In [ ]:
daily_pharm_sales.rolling(window=3, center=True).mean().head()

> Why is there a NaN in the first row? What if we set the parameter `window` to 6?

We can use our index filtering to just look at 2015.

In [ ]:
daily_pharm_sales.rolling(window=7, center=True).mean()['2015'].head()

Instead of plotting the full time series, we can plot the rolling mean, which smooths random changes in sales and removes outliers, helping us identify larger trends.

In [ ]:
daily_pharm_sales.rolling(window=30, center=True).mean().plot();

*Note that rolling doesn't understand if you are missing periods (e.g., if you roll over three days and your data are missing weekends, it'll roll Fri/Sat/Sun), so you need to resample first if you care about that.*

> **Question:** How does the signal that's captured by rolling statistics differ from the signal captured by resampling time series data?

### The Expanding Mean

The expanding mean simply uses all of the data points up to the current time to calculate the mean, as opposed to a moving window.

![](./assets/expanding-mean.jpeg)

##### Calculate and plot the expanding mean and compare it to the rolling mean. Resample by month.

In [ ]:
monthly_pharm_sales = pharm.Sales.resample('M').sum()
monthly_pharm_sales.head()

In [ ]:
rolling_mean = monthly_pharm_sales.rolling(window=3, center=False).mean()
expanding_mean = monthly_pharm_sales.expanding().mean()

In [ ]:
fig, ax = plt.subplots()
rolling_mean.plot(legend = True);
expanding_mean.plot(legend = True);
ax.legend(['Rolling Mean', 'Expanding Mean']);

### Exponentially Weighted Windows

Exponentially weighted windows are one of the most common and effective ways of averaging out noise in time series data. The averaging is done with an "exponential decay" on the contribution of prior means, decreasing the contribution of time points that are further in the past.

The (adjusted) exponentially weighted mean for time, $t$, is defined as:

$$ x_t = \frac{x_t + (1 - \alpha)x_{t-1} + (1 - \alpha)^2x_{t-1} + ... + (1 - \alpha)^{t}x_0} {1 + (1 - \alpha) + (1 - \alpha)^2 + ... + (1 - \alpha)^{t}} $$

Pandas provides the function [`.ewm()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html) to calculate the exponentially weighted mean.


##### Calculate and plot the exponentially weighted sum along with the rolling sum. What's the difference?



In [ ]:
exp_weighted_mean = monthly_pharm_sales.ewm(span=3).mean()

In [ ]:
fig, ax = plt.subplots()
rolling_mean.plot(legend = True);
expanding_mean.plot(legend = True);
exp_weighted_mean.plot(legend = True);
ax.legend(['Rolling Mean', 'Expanding Mean', 'Exponentially Weighted Mean']);

<h2><a id="E">Shifting and Lagging Time Series Data</a></h2>

Another common operation on time series data is shifting or lagging values backward and forward in time. This can help us calculate the percentage of change from sample to sample. Pandas has a `.shift()` method for shifting the data in a DataFrame.

Let's take a look at the Rossman data when we apply lagged features. 

In [ ]:
pharm_store1.head()

Let's shift the sales price by one day.

In [ ]:
shifted_forward = pharm_store1.shift(1)
shifted_forward.head()

*Notice that the first row now contains NaN values because there wasn't a previous day's data to shift to that day.*

Next, let's shift the sales prices by five days.

In [ ]:
shifted_forward5 = pharm_store1.shift(5)
shifted_forward5.head(10)

We can also use negative numbers to shift the sales values in the reverse direction.

In [ ]:
shifted_backward = pharm_store1.shift(-1)
shifted_backward.tail()

Lags can be used to calculate the changes in the values you are tracking with your time series data. In this case, we can use Pandas' `.shift()` method to look at the changes in sales. 

Let's create a new column in our Rossman DataFrame that contains the previous day's sales. 

*Note that we added `.copy()` when we filtered out the store 1 sales earlier. If we didn't explictly tell Pandas that `store1_data` is a copy (instead of a view), we would receive a `SettingCopyWarning`. Here is a useful [video](https://www.youtube.com/watch?v=4R4WsDJ-KVc) that helps explain how to avoid SettingCopyWithWarning errors in Pandas.*

In [ ]:
pharm_store1['Prev Day Sales'] = pharm_store1['Sales'].shift(1)
pharm_store1.head()

Using our new column, it's simple to calculate the one-day change in sales at Store 1. Let's create a new column for this value in our DataFrame as well.

In [ ]:
pharm_store1['Sales Change'] = pharm_store1['Sales'] - pharm_store1['Prev Day Sales']
pharm_store1.head()

> **Question:** What are some other real-world applications you can think of for shifting data in a time series?

**Important Takeaways**
* Trends are long-term changes in data. We have to sort through the noise of a time series to identify trends.
* We can resample the data to look at simple aggregates and identify patterns.
* Rolling statistics give us a local statistic of an average in time, smoothing out random fluctuations and removing outliers.

<h2><a id="F">Independent Practice</a></h2>

**Instructor Note:** These are optional and can be assigned as student practice questions outside of class.

### Load the Unemployment data set. Perform any necessary cleaning and preprocess the data by creating a `datetime` index.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
%matplotlib inline

In [ ]:
unemp = pd.read_csv('./data/unemployment.csv')
unemp.head()

In [ ]:
unemp.tail()

In [ ]:
# Rename columns in the dataset.
unemp.columns = ['year_quarter', 'unemployment_rate']

# Drop % and convert string to float
unemp['unemployment_rate'] = unemp['unemployment_rate'].map(lambda x: float(str(x).replace('%','')))

unemp.head()

In [ ]:
# Drop NaN values.
unemp.dropna(inplace=True)

unemp.tail()

In [ ]:
# This is quarterly data, so converting to datetime is a bit complicated. 
# Use .dt.to_period('Q') to help represent the string as a datetime object.
unemp['date'] = 
# Set Date column to index.


### Plot the unemployment rate.

### Calculate the rolling mean of years with `window=3 `, without centering, and plot both the unemployment rates and the rolling mean data.

In [ ]:
yearly = 
yearly.head()

In [ ]:
fig, ax = plt.subplots()
unemp.unemployment_rate.plot(legend = True);
yearly.plot(legend = True);
ax.legend(['Unemployment Rate', 'Rolling mean of years']);

### Calculate the rolling median with `window=5` and `window=15`. Plot both together with the original data.

In [ ]:
roll_med_5 = 
roll_med_15 = 

In [ ]:
fig, ax = plt.subplots()

#Plot together the original data and both rolling medians.

### Calculate and plot the expanding mean. Resample by quarter. Plot the rolling mean and the expanding mean together.

In [ ]:
date_ticks = unemp.index.to_timestamp()

# Resample by quarter.
quarterly_data = 

# Calculate the rolling mean with window = 1 without centering.
rolling_mean = 

# Calculate the expanding mean.
expanding_mean = 

# Create a plot of the rolling/expanding mean vs date_ticks. Why this way to plot the data is better?
plt.plot(date_ticks, rolling_mean, alpha=1, lw=2, label='Rolling mean')
plt.plot(date_ticks, expanding_mean, alpha=1, lw=2, label='Expanding mean')

plt.legend(loc='upper left')

plt.tick_params(labelsize=14)

### Calculate exponentially weighted sum and add to the previous plot.

In [ ]:
exp_weighted_mean = 

# Plot together the rolling, expanding and exponentially weighted means.